In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pickle
    

In [2]:
# load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# preprocessing
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Encode the categorical variable
label_gender_encoder = LabelEncoder()
df["Gender"] = label_gender_encoder.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# One hot encode Geography column
from sklearn.preprocessing import OneHotEncoder
OHE_geo=OneHotEncoder()
geo_encoder = OHE_geo.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [7]:
OHE_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=OHE_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combine one hot encoded columns with original data
df = pd.concat([df.drop('Geography', axis=1),geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Save the encoders and scaler
with open('label_gender_encoder.pkl','wb') as file:
    pickle.dump(label_gender_encoder,file)

with open('OHE_encoder_geo.pkl', 'wb') as file:
    pickle.dump(OHE_geo,file)

In [11]:
## Divide the data into dependent and independent features
X = df.drop('Exited', axis=1)
y= df['Exited']

## Splitting the data
X_train, X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state=45)

## Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

## ANN implementation

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [14]:
## Build the ANN model 
ann = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)), ## first Hidden layer
    Dense(32, activation = 'relu'), ## Hidden layer 2
    Dense(1, activation ='sigmoid') ## output layer
]
)

In [15]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss= tf.keras.losses.BinaryCrossentropy()

In [17]:
## Compile the model
ann.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
## Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir= "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)


In [19]:
## Set up Early stopping
early_stopping_callback= EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [20]:
## Training the model
history= ann.fit(
    X_train,y_train, validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 0.4021 - accuracy: 0.8328 - val_loss: 0.3690 - val_accuracy: 0.8500
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3601 - accuracy: 0.8540 - val_loss: 0.3546 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3475 - accuracy: 0.8599 - val_loss: 0.3529 - val_accuracy: 0.8560
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3445 - accuracy: 0.8609 - val_loss: 0.3379 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3413 - accuracy: 0.8622 - val_loss: 0.3419 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3352 - accuracy: 0.8633 - val_loss: 0.3363 - val_accuracy: 0.8610
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3343 - accuracy: 0.8650 - val_loss: 0.3510 - val_accuracy: 0.85

In [21]:
ann.save('model.h5')

c:\Users\ashri\Desktop\Prashu\Gen AI\ANN\annenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
## Load tensorboard Extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir=logs/fit20250830-104110

Reusing TensorBoard on port 6006 (pid 3832), started 0:30:34 ago. (Use '!kill 3832' to kill it.)